In [ ]:
from collections import namedtuple
from typing import Any, Dict, List, Sequence, Set, Tuple

def compose_query_string(
    base_column: str,
    additional_columns: List[str],
    intersections: List[str],
    version: str = "latest",
    dataset: str = "gadm__tcl__iso_change",
    api_uri: str = "https://staging-data-api.globalforestwatch.org"
) -> str:
    base_uri = f"{api_uri}/dataset/{dataset}/{version}/query/json?sql="

    base_query_sql: str = (
        "select "
        + ", ".join([
            f"sum({base_column}) as {base_column}",
            *additional_columns
        ])
        + " from data"
    )

    intersections_sql = "where " + " and ".join(i for i in intersections) if intersections else ""

    group_by_sql =  "group by " + ", ".join(c for c in additional_columns) if additional_columns else ""

#     order_by_sql =  "order by " + ", ".join(c for c in additional_columns) if additional_columns else ""

    query_sql = " ".join([base_query_sql, intersections_sql, group_by_sql])  # , order_by_sql])

    return base_uri + query_sql


In [ ]:
SQLQueryArgs = namedtuple("SQLQueryArgs", ["base_column", "additional_columns", "intersections"])


# Base columns (i.e. columns to be summed across rows)
emissions = '"gfw_full_extent_gross_emissions__Mg_CO2e"'    # The quotes are needed to preserve case!
# removals  = '"gfw_full_extent_gross_removals__Mg_CO2"'    # The quotes are needed to preserve case!
# net_flux  = '"gfw_full_extent_net_flux__Mg_CO2e"'         # The quotes are needed to preserve case!
TCL = "umd_tree_cover_loss__ha"
TCLF = "umd_tree_cover_loss_from_fires__ha"

# More column names
drivers__type  = "tsc_tree_cover_loss_drivers__type"
logging_type   = "gfw_planted_forests__type"
TCD_2k__thresh = "umd_tree_cover_density_2000__threshold"
TCL__year      = "umd_tree_cover_loss__year"
WDPA__cat      = "wdpa_protected_areas__iucn_cat"

# Intersections (boolean SQL snippets)
has_drivers_type = f"{drivers__type} IS NOT NULL"
has_logging_type = f"{logging_type} IS NOT NULL"
has_WDPA_cat     = f"{WDPA__cat} IS NOT NULL"

is_AZE            = "is__birdlife_alliance_for_zero_extinction_site = true"
is_IFL_2k         = "is__ifl_intact_forest_landscapes_2000 = true"
is_indigenous     = "is__landmark_indigenous_and_community_lands = true"
is_KBA            = "is__birdlife_key_biodiversity_areas = true"
is_mangroves_1996 = "is__gmw_global_mangrove_extent_1996 = true"
is_mining         = "is__gfw_mining_concessions = true"
is_oil_palm       = "is__gfw_oil_palm = true"
is_plantation     = "is__gfw_managed_forests = true"
is_primary        = "is__umd_regional_primary_forest_2001 = true"
is_wood_fiber     = "is__gfw_wood_fiber = true"


In [ ]:
versions = ["v20221104", "v20230412"]

# Setup the GHG comparisons
columns_of_interest = [
    "iso",
    TCD_2k__thresh,
    TCL__year,
    drivers__type,
    logging_type,
    WDPA__cat
]

GHG_intersections = [
    is_AZE,
    is_IFL_2k,
    is_indigenous,
    is_KBA,
    is_mangroves_1996,
    is_mining,
    is_oil_palm,
    is_plantation,
    is_primary,
    is_wood_fiber
]

# base_column, [additional_columns], [intersections]
GHG_query_args: List[SQLQueryArgs] = [
    SQLQueryArgs(emissions, columns_of_interest, []),
    *[
        SQLQueryArgs(emissions, columns_of_interest, [intersection])
        for intersection in GHG_intersections
    ]
]

# GHG_query_args

In [ ]:
# Setup the TCL/TCLF comparisons

TCL_intersections = [
    has_drivers_type,
    is_IFL_2k,
    is_primary,
    has_WDPA_cat
]

TCLF_intersections = [
    is_AZE,
    is_indigenous,
    is_KBA,
    has_logging_type,
    is_mining,
    is_oil_palm,
    has_WDPA_cat,
    is_wood_fiber
]

TCL_query_args: List[SQLQueryArgs] = [
    # TCL
#     SQLQueryArgs(TCL, columns_of_interest, []),

    # TCL + single intersections
    *[
        SQLQueryArgs(TCL, columns_of_interest, [intersection])
        for intersection in TCL_intersections
    ],

    # TCL + multiple intersections
    SQLQueryArgs(TCL, [columns_of_interest], [is_IFL_2k, has_WDPA_cat]),

    # TCLF
    SQLQueryArgs(TCLF, columns_of_interest, []),

    # TCLF + single intersections
    *[
        SQLQueryArgs(TCLF, columns_of_interest, [intersection])
        for intersection in TCLF_intersections
    ],
    SQLQueryArgs(TCLF, columns_of_interest, [is_IFL_2k]),
    SQLQueryArgs(TCLF, columns_of_interest, [is_plantation]),

    # TCLF + multiple intersections
    *[
        SQLQueryArgs(TCLF, columns_of_interest, [is_IFL_2k, intersection])
        for intersection in TCLF_intersections
    ],
    *[
        SQLQueryArgs(TCLF, columns_of_interest, [is_plantation, intersection])
        for intersection in TCLF_intersections
    ],
]

# TCL_query_args

In [ ]:
import csv

import numpy as np
import pandas as pd
from httpx import AsyncClient

client = AsyncClient()




for filename, query_args in [
#     ("ghg_data.csv", GHG_query_args),
    ("tcl_data.csv", TCL_query_args)
]:
    dfs: List = []

    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, dialect=csv.excel)

        csv_writer.writerow([
            "value", *columns_of_interest, "intersections", versions[0], versions[1], "percent_change"
        ])

#         for combo in query_args[1:2]:  # Run the first query to test
        for combo in query_args:

            value_key = combo.base_column.strip('"')

            req_uri = compose_query_string(*combo, version=versions[0], api_uri="https://data-api.globalforestwatch.org")
            resp = await client.get(req_uri, follow_redirects=True, timeout=60)
            if resp.status_code != 200:
                print(resp.text)
                continue
            data_a = resp.json()["data"]

            df_a = pd.DataFrame(data_a)
            df_a["value"] = value_key
            df_a["intersections"] = f"{' | '.join(combo.intersections)}"

            req_uri = compose_query_string(*combo, version=versions[1], api_uri="https://staging-data-api.globalforestwatch.org")
            resp = await client.get(req_uri, follow_redirects=True, timeout=60)
            if resp.status_code != 200:
                print(resp.text)
                continue
            data_b = resp.json()["data"]

            df_b = pd.DataFrame(data_b)
            df_b["value"] = value_key
            df_b["intersections"] = f"{' | '.join(combo.intersections)}"

            df_a = df_a.rename(columns={value_key: versions[0]})
            df_b = df_b.rename(columns={value_key: versions[1]})

            combined_df = df_a.merge(df_b, on=[*combo.additional_columns, 'intersections', 'value'])
            dfs.append(combined_df)

    final_df = pd.concat(dfs)
    final_df.to_csv(
        filename, 
        index=False,
        float_format = '%.3f',
        columns=['value', 'intersections', *(query_args[0].additional_columns), versions[0], versions[1]]
    )

#     final_df = df_a.merge(df_b, on=['iso', 'umd_tree_cover_loss__year',
#        'umd_tree_cover_density_2000__threshold', 'value'])
#     final_df = pd.concat(dfs)
#     final_df["diff"] = (final_df[versions[0]] - final_df[versions[1]]).abs() / ((final_df[versions[0]] + final_df[versions[1]]) / 2) * 100
#     final_df.to_csv(
#         filename, 
#         index=False,
#         float_format = '%.3f',
#         columns=['value', 'intersections', *(query_args[0].additional_columns), versions[0], versions[1], 'diff']
#     )

        
#             for a, b in zip(data_a, data_b):
#                 row = [
#                     value_key,
#                     *[a[c] for c in combo.additional_columns],
#                     " | ".join(combo.intersections),
#                     a[value_key],
#                     b[value_key],
#                     np.diff([a[value_key], b[value_key]])[0] / a[value_key] * 100
#                 ]
#                 csv_writer.writerow(row)

print("All done!")